## Przygotowanie

Przed rozpoczęciem pracy z notatnikiem proszę zmienić jego nazwę dodając na początku numer albumu, imię i nazwisko.
{nr_albumu}\_{imię}\_{nazwisko}\_{nazwa}

Po wykonaniu wszystkich zadań proszę przesłać wypełniony notatnik przez platformę ELF za pomocą formularza "Prześlij projekt" w odpowiedniej sekcji. 

## Regresja liniowa prosta

Regresja liniowa prosta, to szczególny przypadek regresji liniowej, w którym zmienną objaśnaną przewidujemy za pomocą jednej zmiennej objaśniającej. Zadanie będzie polegało na wyznaczeniu funkcji regresji opisującej zależność zarobków od lat doświadczenia. 

Zbiór danych do tego zadania, to Salary.csv. Znajduje się w katalogu datasets.
W zbiorze danych znajduje się 35 obserwacji. Każdy wpis jest osobną obserwacją. W zbiorze znajdują się 3 kolumny: YearsExperience, Age i Salary. W pierwszym zadaniu należy wykorzystać YearsExperience i Salary, pomijając Age.

### Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

### Zadanie 1

Wczytaj dane z pliku Salary.csv, a następnie stwórz wykres przedstawiający obserwacje.

In [ ]:
data = pd.read_csv("Salary.csv")

fig, axes = plt.subplots(1,2,figsize=(12,8))
axes[0].scatter(x=data["Age"], y=data["Salary"])
axes[0].set_title("Salary vs Age")
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Salary")

axes[1].scatter(x=data["YearsExperience"], y=data["Salary"])
axes[1].set_title("Salary vs Years of Experience")
axes[1].set_xlabel("Years of Experience")
axes[1].set_ylabel("Salary")

plt.tight_layout()
plt.show()

### Zadanie 2

Implementacja algorytmu regresji liniowej prostej.

Żeby dobrze zrozumieć zapis matematyczny, który początkowo może sprawiać problemy, przejdziemy po kolei po elementach składowych algorytmu. Następnie złączymy elementy w całość.

Wzór na regresję liniową w naszym przypadku będzie wyglądał następująco:

\begin{equation}
f(x^{(i)}) = \beta_{0} + \beta_{1}x_1 = \beta_{0} + \beta_{1}  YearsExperience
\end{equation}

Przypomnijmy, że zapis $x^{(i)}$ oznacza wektor dla $i$-tej obserwacji. W naszym przypadku ten wektor będzie zawierał tylko $1$ wartość dla cechy $YearsExperience$.


_Uwaga: W różnych źródłach algorytm regresji liniowej ma różne zapisy. Czasem podawane są wzory w postaci z sumą, czasem w postaci macierzowej. Jest to spowodowane tym, że algorytm można zaimplementować na te dwa sposoby. Łatwiejszym i bardziej intuicyjnym podejściem jest podejście z sumą, która bezpośrednio sugeruje wykokrzystanie pętli w celu iteracji po obserwacjach/cechach. Implementacja z wykorzystaniem macierzy jest zwykle krótsza i "bardziej elegancka", ale również bardziej wydajna. Aby dobrze zrozumieć działanie algorytmu, najlepiej jest zaimplementować obie wersje i porównać je ze sobą._


#### 2.1 Inicjalizacja współczynników $\beta$ regresji

Pierwszym krokiem jest inicjalizacja współczynników regresji. W przypadku regresji liniowej prostej mamy dwa współczynniki $\beta_{0}$ i $\beta_{1}$. Stwórz dwie zmienne będące współczynnikami regresji liniowej prostej i zainicjalizuj je losowymi wartościami z przedziału $(0,1)$.

Dodatkowo stwórz zmienną *alpha*, która przyjmie wartość od $(0,1)$. Możesz ustawić ją ręcznie i sprawdzać jak różne wartości mają wpływ na regresję. 

In [ ]:
import random
def initialize_coefficients() -> Tuple[float, float, float]:
    return random.random(),random.random(),random.random()

#### 2.2 Obliczenie predykcji

Kolejnym krokiem jest obliczenie wartości funkcji regresji dla wszystkich obserwacji w zbiorze danych. Jest to po prostu wstawienie kolejnych wartości pod wzrór regresji.

\begin{equation}
f(x) = \beta_{0} + \beta_{1}x_1
\end{equation}

Można zrobić to z wykorzystaniem operacji na macierzach (wektorach), albo z wykorzystaniem klasycznej iteracji.

In [ ]:
def calculate_regression_function(x: np.ndarray, beta0: float, beta1: float) -> np.ndarray:
    return beta0 + beta1 * x

#### 2.3 Obliczenie błędu

Obliczenie wartości błędu regresji nie jest konieczne do aktualizacji wag, jednak jest to bardzo cenna informacja czy nasz algorytm działa poprawnie. Wartość błędu nie może rosnąć w kolejnych epokach.

Błąd należy obliczyć zgodnie ze wzorem:

\begin{equation}
    SSR = \frac{1}{2m} \sum_{i=1}^{m}(f(x^{(i)}) - y^{(i)})^2
\end{equation}

In [ ]:
def calculate_error(predictions: np.ndarray, y: np.ndarray, beta0: float, beta1: float) -> float:
    m = y.shape[0]
    return (np.sum((predictions - y)**2))/(2*m)

#### 2.4 Obliczenie gradientu

Żeby obliczyć gradient, należy obliczyć pochodne cząstkowe względem parametrów $\beta_{0}$ i $\beta_{1}$.

\begin{equation}
    \frac{\partial SSR}{\partial \beta_{0}} = \frac{1}{m} \sum^{m}_{i=1} (f(x^{(i)}) - y^{(i)})
\end{equation}

\begin{equation}
    \frac{\partial SSR}{\partial \beta_{1}} = \frac{1}{m} \sum^{m}_{i=1} (f(x^{(i)}) - y^{(i)})x_{1}^{(i)}
\end{equation}

Tutaj ponownie jak wcześniej można wykorzystać operacje na macierzach, lub iteracyjnie obliczyć sumę.

In [ ]:
def calculate_gradient(predictions: np.ndarray, y: np.ndarray, x: np.ndarray, beta0: float, beta1: float) -> Tuple[float, float]:
    m = y.shape[0]
    diff = predictions - y
    beta0_gradient = (np.sum(diff))/m
    beta1_gradient = (np.sum(diff * x))/m
    return beta0_gradient, beta1_gradient

####  2.5 Aktualizacja współczynników regresji (wag)

Po obliczeniu pochodnych cząstkowych należy obliczyć nowe wartości dla współczynników regresji.


\begin{equation}
    \beta_{0} = \beta_{0} - \alpha \frac{\partial SSR}{\partial \beta_{0}}
\end{equation}

\begin{equation}
    \beta_{1} = \beta_{1} - \alpha \frac{\partial SSR}{\partial \beta_{1}}
\end{equation}

In [ ]:
def update_regression_coefficients(x: np.ndarray, y: np.ndarray, beta0: float, beta1: float, alpha: float) -> Tuple[float, float]:
    gradient_0, gradient_1 = calculate_gradient(
        calculate_regression_function(x,beta0,beta1),
        y,
        x,
        beta0,
        beta1)

    new_beta0 = beta0 - alpha * gradient_0
    new_beta1 = beta1 - alpha * gradient_1
    return new_beta0, new_beta1

#### 2.6 Finalna wersja algorytmu

Powyższe działania, to wszystkie elementy potrzebne do stworzenia algorytmu regresji liniowej prostej. Jeden cykl takich operacji nazywany jest **epoką**. Idea obliczania współczynników regresji z wykorzystaniem gradientu polega na iteracyjnym aktualizowaniu współczynników do momentu, aż błąd przestanie znacznie się zmieniać. Można również ustawić jakaś stałą ilość epok. W każdej epoce wykorzystuje się ponownie ten sam zestaw danych.

Skoro wiadomo już jakie pojedyncze etapy należy wykonać, żeby obliczyć regresję liniową prostą, przyszedł czas na zebranie wszystkiego w jednym miejscu.

Proszę zaimplementować funkcję `learn_and_fit(x, y)`, która dla danych wejściowych będzie zwracać współczynniki regresji w każdej z epok. Dodatkowo proszę zwracać również błąd regresji w każdej epoce. Funkcja może być zaimplementowana w dowolny sposób. Może bezpośrednio zawierać wszystkie instrukcje, może korzystać z innych funkcji pomocniczych albo może korzystać z klasy reprezentującą regresję liniową prostą.

Na końcu notebooka znajduje się test jednostkowy, który musi przechodzić przy prawidłowej implementacji algorytmu.

In [ ]:
'''
input:
x - wartości zmiennej objaśniającej YearsExperience dla wszystkich obserwacji
y - wartości zmiennej objaśnianej Salary dla wszystkich obserwacji

output:
b0: [] - lista z współczynnikami beta_0 w każdej z epok
b1: [] - lista z współczynnikami beta_1 w każdej z epok
error: [] - lista z błędem w każdej epoce
'''
def learn_and_fit(x: np.ndarray, y: np.ndarray, alpha=0.01, epochs = 50) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    # standarization
    x = (x - np.mean(x)) / np.std(x)
    y = (y - np.mean(y)) / np.std(y)

    errors = []
    b0 = []
    b1 = []

    beta0 , beta1, _ = initialize_coefficients()

    for i in range(epochs) :
        predictions = calculate_regression_function(x, beta0, beta1)
        error = calculate_error(predictions, y, beta0, beta1)
        errors.append(error)
        beta0, beta1 = update_regression_coefficients(x, y, beta0, beta1, alpha)
        b0.append(beta0)
        b1.append(beta1)
        if error < 0.05:
            return np.array(b0), np.array(b1), np.array(errors)

    return np.array(b0), np.array(b1), np.array(errors)

### Zadanie 3

Stwórz wykres zmiany błędu regresji (oś Y) względem epoki (oś X)

In [ ]:
data = pd.read_csv("Salary.csv")

x = data["YearsExperience"]
y = data["Salary"]

epochs = 300
_,_,errors = learn_and_fit(x, y, alpha=0.01 ,epochs=epochs)

plt.figure(figsize=(10, 6))
plt.plot(range(len(errors)), errors, marker='o', color='blue')
plt.title("Regression error during epochs")
plt.xlabel("Epoch")
plt.ylabel("Regression error")
plt.grid(True)
plt.tight_layout()
plt.show()

### Zadanie 4

Do wykresu stworzonego w zadaniu 1 dodaj prostą regresji. Stwórz 3 wykresy przedstawiające jak zmieniała się funkcja regresji na przestrzeni epok (pierwsza, środkowa, ostatnia epoka).

In [ ]:
x = data["YearsExperience"].values
y = data["Salary"].values


b0, b1, _ = learn_and_fit(x, y, alpha=0.01, epochs=200)

x = (x - np.mean(x)) / np.std(x)
y = (y - np.mean(y)) / np.std(y)

beta0 = b0[-1]
beta1 = b1[-1]

x_line = np.linspace(min(x), max(x), 100)

first = 0
middle = len(b0) // 2
last = -1

# Obliczanie wartości y dla każdej prostej regresji
y_first = b0[first] + b1[first] * x_line
y_middle = b0[middle] + b1[middle] * x_line
y_last = b0[last] + b1[last] * x_line

plt.figure(figsize=(10,6))
plt.scatter(x, y, color='blue', label='Real data')

plt.plot(x_line, y_first, label=f"Epoch {first+1}", color="red")
plt.plot(x_line, y_middle, label=f"Epoch {middle+1}", color="green")
plt.plot(x_line, y_last, label=f"Epoch {len(b0)}", color="blue")

plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.title('Linear regression - fit line')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Zadanie 5

Wybierz dowolnie trzy różne wartości współczynnika $\alpha$. Ile epok mu zostać użytych żeby otrzymać zamierzoną wartość błędu?

In [ ]:
alpha_list = [0.001, 0.01, 0.1]

for alpha in alpha_list:
    b0, b1, errors = learn_and_fit(x, y, alpha=alpha, epochs=1000)


    plt.figure(figsize=(10, 6))
    plt.plot(range(len(errors)), errors, marker='o', color='blue')
    plt.title(f"Regression error during epochs - alpha = {alpha}")
    plt.xlabel("Epoch")
    plt.ylabel("Regression error")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### Testy jednostkowe

In [ ]:
import unittest
import pandas as pd

class SimpleLinearRegressionTest(unittest.TestCase):
    
    def test_learn_and_fit(self):
        df = pd.read_csv('Salary.csv', sep=',')
        x = df['YearsExperience'].values.reshape(df['YearsExperience'].shape[0], 1)
        y = df['Salary'].values.reshape(df['Salary'].shape[0], 1)
        
        b0, b1, error = learn_and_fit(x, y)
        
        self.assertTrue(len(b0) > 1)
        self.assertTrue(len(b1) > 1)
        self.assertTrue(len(b0) == len(b1))
        self.assertTrue(all(i >= j for i, j in zip(error, error[1:]))) #Sprawdzenie, czy błędy nie rosną
        
unittest.main(argv=[''], verbosity=2, exit=False)

### Zadanie 6

Stwórz test jednostkowy sprawdzający czy funkcja inicjalizująca współczynniki regresji zwraca wartości z przedziału (0, 1)

In [ ]:
import unittest

class SimpleLinearRegressionTest(unittest.TestCase):

    def test_initialize_coefficients(self):

        beta0, beta1, alpha = initialize_coefficients()


        self.assertTrue(isinstance(beta0,float))
        self.assertTrue(isinstance(beta1,float))
        self.assertTrue(isinstance(alpha,float))
        self.assertTrue(0 < beta0 < 1)
        self.assertTrue(0 < beta1 < 1)
        self.assertTrue(0 < alpha < 1)

unittest.main(argv=[''], verbosity=2, exit=False)